<a href="https://colab.research.google.com/github/pravallikai/AarogyaSetu_Android/blob/master/meta_prompt_maker_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir meta_prompt_maker


In [2]:
%cd meta_prompt_maker

/content/meta_prompt_maker


In [3]:
!mkdir app

In [4]:
!mkdir app/templates

In [5]:
%%writefile requirements.txt
# FastAPI is the web framework
fastapi==0.115.6

# Uvicorn runs the FastAPI server
uvicorn[standard]==0.34.0

# HTTP client for APIs
httpx==0.28.1

# Loads environment variables safely
python-dotenv==1.0.1

# HTML template engine
jinja2==3.1.4


Writing requirements.txt


In [6]:
%%writefile README.md
# Meta Prompt Maker

Paste raw text and get a world-class, structured prompt.

## Tech
- FastAPI
- NVIDIA API (primary)
- OpenRouter API (fallback)

## Run locally
uvicorn app.main:app --reload

## Deploy
Ready for Render


Writing README.md


In [7]:
%%writefile app/prompt_builder.py
# This function builds a clean, structured prompt from raw user text
def build_structured_prompt(user_text: str) -> str:
    # Remove extra spaces from the user input
    cleaned_text = user_text.strip()

    # Create the structured prompt using a formatted multi-line string
    structured_prompt = f"""
---
## Task Definition
You are an expert prompt engineer.
Your task is to convert the user's raw input into a clear, high-quality prompt
that another powerful AI model can execute accurately.

## Context
- Original user input:
  "{cleaned_text}"

- Assumptions:
  - If details are missing, clearly state reasonable assumptions.
  - Do not invent facts.
  - Ask clarification questions only if absolutely required.

- Style requirements:
  - Clear
  - Structured
  - Professional
  - Easy to understand

## References
- If the task involves writing (email, report, post), include a short example format.
- If the task involves data or files, ask the user to provide them.
- If the task involves reasoning, outline logical steps.

## Evaluation Criteria
- The output must fully match the user's intent.
- The instructions must be unambiguous.
- The output must be ready to use with minimal edits.

## Iteration & Refinement
- If the user wants changes, regenerate with:
  - Different tone
  - Different length
  - More or less detail
- Always improve clarity before adding complexity.
---
"""

    # Return the final structured prompt
    return structured_prompt


Writing app/prompt_builder.py


In [8]:
%%writefile app/llm_clients.py
# Import httpx to make HTTP requests to AI APIs
import httpx

# Import os to read environment variables (API keys)
import os

# This function sends the prompt to NVIDIA API
async def call_nvidia(prompt: str) -> str:
    # Get NVIDIA API key from environment variables
    nvidia_api_key = os.getenv("NVIDIA_API_KEY")

    # If NVIDIA key is missing, stop and raise an error
    if not nvidia_api_key:
        raise RuntimeError("NVIDIA API key not found")

    # Define NVIDIA API endpoint (placeholder – user fills real endpoint)
    url = "https://api.nvidia.com/v1/chat/completions"

    # Prepare headers needed for authorization
    headers = {
        "Authorization": f"Bearer {nvidia_api_key}",  # Proves who we are
        "Content-Type": "application/json"             # We send JSON data
    }

    # Prepare request body for the AI model
    payload = {
        "model": "nvidia-model-name",                   # User sets model name
        "messages": [
            {"role": "user", "content": prompt}         # User prompt
        ]
    }

    # Send request to NVIDIA API
    async with httpx.AsyncClient() as client:
        response = await client.post(url, headers=headers, json=payload)

    # If NVIDIA API fails, raise an error
    if response.status_code != 200:
        raise RuntimeError("NVIDIA API request failed")

    # Convert response to JSON
    data = response.json()

    # Return the generated text from NVIDIA response
    return data["choices"][0]["message"]["content"]


# This function sends the prompt to OpenRouter API (fallback)
async def call_openrouter(prompt: str) -> str:
    # Get OpenRouter API key from environment variables
    openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

    # If OpenRouter key is missing, stop and raise an error
    if not openrouter_api_key:
        raise RuntimeError("OpenRouter API key not found")

    # Define OpenRouter API endpoint
    url = "https://openrouter.ai/api/v1/chat/completions"

    # Prepare headers for OpenRouter authentication
    headers = {
        "Authorization": f"Bearer {openrouter_api_key}", # Proves who we are
        "Content-Type": "application/json"                # JSON format
    }

    # Prepare request body
    payload = {
        "model": "openrouter/model-name",                 # User chooses model
        "messages": [
            {"role": "user", "content": prompt}           # User prompt
        ]
    }

    # Send request to OpenRouter
    async with httpx.AsyncClient() as client:
        response = await client.post(url, headers=headers, json=payload)

    # If OpenRouter API fails, raise an error
    if response.status_code != 200:
        raise RuntimeError("OpenRouter API request failed")

    # Convert response to JSON
    data = response.json()

    # Return the generated text from OpenRouter response
    return data["choices"][0]["message"]["content"]


# This function decides which API to use
async def generate_with_fallback(prompt: str) -> tuple[str, str]:
    # First, try NVIDIA API
    try:
        result = await call_nvidia(prompt)
        return result, "NVIDIA"
    except Exception:
        pass  # If NVIDIA fails, move to OpenRouter

    # If NVIDIA fails, try OpenRouter
    try:
        result = await call_openrouter(prompt)
        return result, "OpenRouter"
    except Exception:
        pass  # If OpenRouter also fails, continue

    # If both APIs fail, return an error message
    return "All APIs failed. Please check API keys or limits.", "None"


Writing app/llm_clients.py


In [9]:
%%writefile app/main.py
# Import FastAPI to create the web application
from fastapi import FastAPI, Request

# Import HTMLResponse so we can return HTML pages
from fastapi.responses import HTMLResponse, JSONResponse

# Import Jinja2Templates to render HTML templates
from fastapi.templating import Jinja2Templates

# Import StaticFiles to serve static files like CSS (if needed later)
from fastapi.staticfiles import StaticFiles

# Import our prompt builder logic
from app.prompt_builder import build_structured_prompt

# Import our API fallback logic (NVIDIA → OpenRouter)
from app.llm_clients import generate_with_fallback

# Create the FastAPI application
app = FastAPI(title="Meta Prompt Maker")

# Tell FastAPI where HTML templates are stored
templates = Jinja2Templates(directory="app/templates")

# (Optional) Mount static folder for future CSS/JS
app.mount("/static", StaticFiles(directory="app"), name="static")


# This route shows the main web page
@app.get("/", response_class=HTMLResponse)
async def home(request: Request):
    # Render index.html and pass request object
    return templates.TemplateResponse(
        "index.html",
        {"request": request}
    )


# This API route generates a structured prompt
@app.post("/api/generate")
async def generate_prompt(payload: dict):
    # Extract text sent by the user
    user_text = payload.get("text", "").strip()

    # If user did not send text, return an error
    if not user_text:
        return JSONResponse(
            {"error": "Please enter some text."},
            status_code=400
        )

    # Build the structured prompt locally
    structured_prompt = build_structured_prompt(user_text)

    # Send prompt to NVIDIA → OpenRouter fallback
    result, provider = await generate_with_fallback(structured_prompt)

    # Return the final result to the UI
    return {
        "provider": provider,
        "prompt": result
    }


# This API route regenerates the prompt
@app.post("/api/regenerate")
async def regenerate_prompt(payload: dict):
    # Reuse the same generation logic
    return await generate_prompt(payload)


Writing app/main.py


In [10]:
%%writefile app/templates/index.html
<!DOCTYPE html>
<html>
<head>
    <!-- Set character encoding -->
    <meta charset="UTF-8">

    <!-- Page title shown in browser tab -->
    <title>Meta Prompt Maker</title>

    <!-- Make page responsive on all devices -->
    <meta name="viewport" content="width=device-width, initial-scale=1">

    <!-- Simple styling for clean UI -->
    <style>
        body {
            font-family: Arial, sans-serif;   /* Easy-to-read font */
            max-width: 900px;                 /* Limit width */
            margin: 30px auto;                /* Center page */
            padding: 10px;
        }
        textarea {
            width: 100%;                      /* Full width */
            height: 180px;                    /* Big input box */
            padding: 10px;
        }
        button {
            padding: 10px 15px;               /* Button size */
            margin-top: 10px;
            cursor: pointer;
        }
        pre {
            background: #f5f5f5;              /* Light background */
            padding: 15px;
            white-space: pre-wrap;            /* Wrap text */
            border-radius: 8px;
            margin-top: 15px;
        }
        .provider {
            margin-top: 10px;
            font-size: 14px;
            color: #555;
        }
    </style>
</head>

<body>

    <!-- App heading -->
    <h2>Meta Prompt Maker</h2>

    <!-- Text area for user input -->
    <textarea id="inputText" placeholder="Paste your raw text here..."></textarea>

    <!-- Generate button -->
    <button onclick="generatePrompt()">Generate</button>

    <!-- Regenerate button -->
    <button onclick="regeneratePrompt()">Regenerate</button>

    <!-- Shows which API was used -->
    <div class="provider" id="providerInfo"></div>

    <!-- Output area -->
    <pre id="outputArea">Your structured prompt will appear here.</pre>

    <!-- JavaScript logic -->
    <script>
        // Call backend API to generate prompt
        async function generatePrompt() {
            const text = document.getElementById("inputText").value;

            const response = await fetch("/api/generate", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({ text: text })
            });

            const data = await response.json();

            if (data.error) {
                alert(data.error);
                return;
            }

            document.getElementById("providerInfo").innerText =
                "Generated using: " + data.provider;

            document.getElementById("outputArea").innerText =
                data.prompt;
        }

        // Regenerate simply calls generate again
        async function regeneratePrompt() {
            await generatePrompt();
        }
    </script>

</body>
</html>


Writing app/templates/index.html
